In [1]:
from rdflib import Graph,URIRef
import rdflib as rdf
import pandas as pd

rdf_file = "data/aifbfixed_complete.n3"
graph = Graph()
graph.parse(rdf_file, format='n3')
print("Number of triples in graph:", len(graph))

Number of triples in graph: 29226


In [2]:
# data = {}
# for s, p, o in graph:
#     if str(s) not in data:
#         data[str(s)] = {}
#     data[str(s)][p] = str(o)

from collections import defaultdict

data = defaultdict(dict)
for s, p, o in graph:
    data[str(s)][p] = str(o)

In [3]:
data

defaultdict(dict,
            {'http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id1282instance': {rdflib.term.URIRef('http://swrc.ontoware.org/ontology#volume'): '57',
              rdflib.term.URIRef('http://swrc.ontoware.org/ontology#note'): 'see the special issue for more contributions related to the Semantic Web',
              rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): 'http://swrc.ontoware.org/ontology#Article',
              rdflib.term.URIRef('http://swrc.ontoware.org/ontology#journal'): 'Information, Wissenschaft und Praxis',
              rdflib.term.URIRef('http://swrc.ontoware.org/ontology#year'): '2006',
              rdflib.term.URIRef('http://swrc.ontoware.org/ontology#hasProject'): 'http://www.aifb.uni-karlsruhe.de/Projekte/viewProjektOWL/id50instance',
              rdflib.term.URIRef('http://swrc.ontoware.org/ontology#isAbout'): 'http://www.aifb.uni-karlsruhe.de/Forschungsgebiete/viewForschungsgebietOWL/id71instance',
    

In [4]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.fillna('')
df.index = df.index.astype(str)
len(df)

2829

In [5]:
df

,http://swrc.ontoware.org/ontology#volume,http://swrc.ontoware.org/ontology#note,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://swrc.ontoware.org/ontology#journal,http://swrc.ontoware.org/ontology#year,http://swrc.ontoware.org/ontology#hasProject,http://swrc.ontoware.org/ontology#isAbout,http://swrc.ontoware.org/ontology#author,http://swrc.ontoware.org/ontology#title,http://swrc.ontoware.org/ontology#number,...,http://swrc.ontoware.org/ontology#howpublished,http://www.w3.org/2002/07/owl#onProperty,http://www.w3.org/2002/07/owl#allValuesFrom,http://swrc.ontoware.org/ontology#type,http://swrc.ontoware.org/ontology#finances,http://swrc.ontoware.org/ontology#editor,http://swrc.ontoware.org/ontology#isbn,http://swrc.ontoware.org/ontology#edition,http://www.w3.org/2002/07/owl#inverseOf,http://www.w3.org/2000/01/rdf-schema#range
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id1282instance,57,see the special issue for more contributions r...,http://swrc.ontoware.org/ontology#Article,"Information, Wissenschaft und Praxis",2006,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontologies on Demand? - A Description of the S...,6-7,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id454instance,16,Special Issue on Semantic Web,http://swrc.ontoware.org/ontology#Publication,IEEE Intelligent Systems,2001,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontology Learning for the Semantic Web,2,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id391instance,18,,http://swrc.ontoware.org/ontology#Publication,"IEEE Intelligent Systems, Trends & Controversies",2003,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,"Web services: Been there, Done that?",1,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id377instance,96,,http://swrc.ontoware.org/ontology#Publication,,2003,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Annotation of the Shallow and the Deep Web,,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id76instance,17,Trends & Controversies,http://swrc.ontoware.org/ontology#Publication,IEEE Intelligent Systems,2002,,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Emergent Semantics,1,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id141instance,,,http://swrc.ontoware.org/ontology#Publication,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#chapter,,,http://www.w3.org/2002/07/owl#DatatypeProperty,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#author,,,http://www.w3.org/2002/07/owl#ObjectProperty,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#howpublished,,,http://www.w3.org/2002/07/owl#DatatypeProperty,,,,,,,,...,,,,,,,,,,


In [6]:
from rdflib import Graph, URIRef
from collections import defaultdict
import pandas as pd

# Load RDF file
rdf_file = "data/aifbfixed_complete.n3"
graph = Graph()
graph.parse(rdf_file, format='n3')
print("Number of triples in graph:", len(graph))

# Step 1: Build dictionary: subject -> { predicate: [objects] }
data = defaultdict(lambda: defaultdict(list))

for s, p, o in graph:
    data[str(s)][str(p)].append(str(o))  # Store multiple objects per predicate

# Step 2: Convert to flat table (concatenating multiple values into strings)
flattened_data = {}
for subject, predicates in data.items():
    flat_row = {}
    for pred_uri, obj_list in predicates.items():
        # Use short predicate names
        pred_short = pred_uri.split('#')[-1] if '#' in pred_uri else pred_uri.split('/')[-1]
        flat_row[pred_short] = '; '.join(sorted(set(obj_list)))  # Join multiple values
    flattened_data[subject] = flat_row

# Step 3: Convert to DataFrame
df = pd.DataFrame.from_dict(flattened_data, orient='index')
df = df.fillna('')  # Fill missing values with ''
df.index.name = 'subject'

print(f"Final DataFrame shape: {df.shape}")
df


Number of triples in graph: 29226
Final DataFrame shape: (2829, 45)


,volume,note,type,journal,year,hasProject,isAbout,author,title,number,...,head,howpublished,allValuesFrom,onProperty,finances,editor,isbn,edition,inverseOf,range
subject,,,,,,,,,,,,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id1282instance,57,see the special issue for more contributions r...,http://swrc.ontoware.org/ontology#Article; htt...,"Information, Wissenschaft und Praxis",2006,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontologies on Demand? - A Description of the S...,6-7,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id454instance,16,Special Issue on Semantic Web,http://swrc.ontoware.org/ontology#Article; htt...,IEEE Intelligent Systems,2001,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontology Learning for the Semantic Web,2,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id391instance,18,,http://swrc.ontoware.org/ontology#Article; htt...,"IEEE Intelligent Systems, Trends & Controversies",2003,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,"Web services: Been there, Done that?",1,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id377instance,96,,http://swrc.ontoware.org/ontology#InCollection...,,2003,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Annotation of the Shallow and the Deep Web,,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id76instance,17,Trends & Controversies,http://swrc.ontoware.org/ontology#Article; htt...,IEEE Intelligent Systems,2002,,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Emergent Semantics,1,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id141instance,,,http://swrc.ontoware.org/ontology#Publication,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#chapter,,,http://www.w3.org/2002/07/owl#DatatypeProperty,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#author,,,http://www.w3.org/2002/07/owl#ObjectProperty,,,,,,,,...,,,,,,,,,,


In [7]:
df.columns

Index(['volume', 'note', 'type', 'journal', 'year', 'hasProject', 'isAbout',
       'author', 'title', 'number', 'month', 'pages', 'booktitle', 'address',
       'abstract', 'worksAtProject', 'fax', 'photo', 'name', 'affiliation',
       'phone', 'publication', 'isWorkedOnBy', 'dealtWithIn', 'homepage',
       'series', 'chapter', 'subClassOf', 'projectInfo', 'financedBy',
       'member', 'carriedOutBy', 'carriesOut', 'publishes', 'employs', 'head',
       'howpublished', 'allValuesFrom', 'onProperty', 'finances', 'editor',
       'isbn', 'edition', 'inverseOf', 'range'],
      dtype='object')

In [8]:
df['affiliation'].value_counts()

affiliation
                                                                                          2651
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance      73
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance      60
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance      28
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id4instance      16
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance       1
Name: count, dtype: int64

In [9]:
df

,volume,note,type,journal,year,hasProject,isAbout,author,title,number,...,head,howpublished,allValuesFrom,onProperty,finances,editor,isbn,edition,inverseOf,range
subject,,,,,,,,,,,,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id1282instance,57,see the special issue for more contributions r...,http://swrc.ontoware.org/ontology#Article; htt...,"Information, Wissenschaft und Praxis",2006,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontologies on Demand? - A Description of the S...,6-7,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id454instance,16,Special Issue on Semantic Web,http://swrc.ontoware.org/ontology#Article; htt...,IEEE Intelligent Systems,2001,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,Ontology Learning for the Semantic Web,2,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id391instance,18,,http://swrc.ontoware.org/ontology#Article; htt...,"IEEE Intelligent Systems, Trends & Controversies",2003,,,http://www.aifb.uni-karlsruhe.de/Personen/view...,"Web services: Been there, Done that?",1,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id377instance,96,,http://swrc.ontoware.org/ontology#InCollection...,,2003,http://www.aifb.uni-karlsruhe.de/Projekte/view...,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Annotation of the Shallow and the Deep Web,,...,,,,,,,,,,
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id76instance,17,Trends & Controversies,http://swrc.ontoware.org/ontology#Article; htt...,IEEE Intelligent Systems,2002,,http://www.aifb.uni-karlsruhe.de/Forschungsgeb...,http://www.aifb.uni-karlsruhe.de/Personen/view...,Emergent Semantics,1,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.aifb.uni-karlsruhe.de/Publikationen/viewPublikationOWL/id141instance,,,http://swrc.ontoware.org/ontology#Publication,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#chapter,,,http://www.w3.org/2002/07/owl#DatatypeProperty,,,,,,,,...,,,,,,,,,,
http://swrc.ontoware.org/ontology#author,,,http://www.w3.org/2002/07/owl#ObjectProperty,,,,,,,,...,,,,,,,,,,


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd


positive_group_uri = 'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance'
df['label'] = (df['affiliation'] == positive_group_uri).astype(int)

X = df.drop(columns=['affiliation', 'label'], errors='ignore')
y = df['label']

X_encoded = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42,class_weight='balanced')
clf.fit(X_train, y_train)

print("for instance1")
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


for instance1
Accuracy: 0.9840989399293286
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       551
           1       0.88      0.47      0.61        15

    accuracy                           0.98       566
   macro avg       0.93      0.73      0.80       566
weighted avg       0.98      0.98      0.98       566



In [ ]:
#binary classification

import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


warnings.filterwarnings("ignore", category=UserWarning)


instance5_uri = 'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance'
df = df[df['affiliation'] != instance5_uri]
print("Instance 5 was dropped.")

X = df.drop(columns=['affiliation'], errors='ignore')
X_encoded = pd.get_dummies(X)
X_encoded.columns = X_encoded.columns.str.replace(r'[\[\]<]', '_', regex=True)


group_uris = df['affiliation'].unique()


for group_uri in group_uris:
    if not group_uri or pd.isnull(group_uri):
        print("Skipping an empty or invalid group URI.")
        continue

    print(f"\n--- Binary Classification for group: {group_uri} ---")

    
    y = (df['affiliation'] == group_uri).astype(int)

   
    if y.sum() == 0 or y.sum() == len(y):
        print("⚠️ Skipping this class because it does not have both positive and negative samples.")
        continue

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=0.2, random_state=42, stratify=y
    )

 
    clf = XGBClassifier(random_state=42, eval_metric='logloss')  # `use_label_encoder` removed (deprecated)
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))


Instance 5 was dropped.
Skipping an empty or invalid group URI.

--- Binary Classification for group: http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance ---
Accuracy: 0.9893992932862191
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       560
           1       0.50      0.17      0.25         6

    accuracy                           0.99       566
   macro avg       0.75      0.58      0.62       566
weighted avg       0.99      0.99      0.99       566


--- Binary Classification for group: http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance ---
Accuracy: 0.9858657243816255
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       554
           1       0.83      0.42      0.56        12

    accuracy                           0.99       566
   macro avg       0.91      0.71      0.77       566
weighted avg       0.

In [ ]:
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore", category=UserWarning)


instance5_uri = 'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance'
df_multi = df[df['affiliation'] != instance5_uri].copy()
print("Instance 5 was dropped for multiclass classification.")

# Drop rows with empty or null affiliations
df_multi = df_multi[df_multi['affiliation'].notna() & (df_multi['affiliation'] != '')]

# Encode the target labels
label_encoder = LabelEncoder()
df_multi['encoded_label'] = label_encoder.fit_transform(df_multi['affiliation'])

# Prepare features
X = df_multi.drop(columns=['affiliation', 'encoded_label'], errors='ignore')
X_encoded = pd.get_dummies(X)

# Sanitize column names to avoid issues with XGBoost
X_encoded.columns = X_encoded.columns.str.replace('[', '_', regex=False)
X_encoded.columns = X_encoded.columns.str.replace(']', '_', regex=False)
X_encoded.columns = X_encoded.columns.str.replace('<', '_', regex=False)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    df_multi['encoded_label'],
    test_size=0.2,
    random_state=42,
    stratify=df_multi['encoded_label']
)
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight('balanced', y_train)


# Train XGBoost classifier
clf = XGBClassifier(
    random_state=42,
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss'
)

clf.fit(X_train, y_train,sample_weight=sample_weights)

# Evaluate
y_pred = clf.predict(X_test)
print("\nMulticlass Classification Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))


In [55]:
X_train.shape

(141, 680)

In [56]:
y_train.shape

(141,)

In [57]:
X_test.shape

(36, 680)

In [60]:
y_test.shape

(36,)

In [61]:
train_labels_df = pd.read_csv('data/trainingSet.tsv', sep='\t')
test_labels_df = pd.read_csv('data/testSet.tsv', sep='\t')

In [ ]:
train_labels_df.shape

(140, 3)

In [66]:
train_labels_df


,person,id,label_affiliation
0,http://www.aifb.uni-karlsruhe.de/Personen/view...,2.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
1,http://www.aifb.uni-karlsruhe.de/Personen/view...,3.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
2,http://www.aifb.uni-karlsruhe.de/Personen/view...,4.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
3,http://www.aifb.uni-karlsruhe.de/Personen/view...,5.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
4,http://www.aifb.uni-karlsruhe.de/Personen/view...,6.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
...,...,...,...
135,http://www.aifb.uni-karlsruhe.de/Personen/view...,171.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
136,http://www.aifb.uni-karlsruhe.de/Personen/view...,173.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
137,http://www.aifb.uni-karlsruhe.de/Personen/view...,174.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...
138,http://www.aifb.uni-karlsruhe.de/Personen/view...,175.0,http://www.aifb.uni-karlsruhe.de/Forschungsgru...


In [54]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on training and test sets
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Accuracy
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy :", accuracy_score(y_test, y_test_pred))

# Classification report
print("\nTrain Classification Report:")
print(classification_report(y_train, y_train_pred, target_names=label_encoder.classes_, zero_division=0))

print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_, zero_division=0))


Train Accuracy: 0.9361702127659575
Test Accuracy : 0.8333333333333334

Train Classification Report:
                                                                                        precision    recall  f1-score   support

http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance       1.00      1.00      1.00        58
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance       0.75      0.95      0.84        22
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance       0.98      0.83      0.90        48
http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id4instance       0.93      1.00      0.96        13

                                                                              accuracy                           0.94       141
                                                                             macro avg       0.91      0.95      0.93       141
  